# U.S. Geological Survey Class GW3099
Advanced Modeling of Groundwater Flow (GW3099)\
Boise, Idaho\
September 16 - 20, 2024

![title](../../images/ClassLocation.jpg)

# NetCDF configuration package and visualization

Additional NetCDF configuration options are available in the `NCF6` package.  These include a `WKT` (well-known text) string that can aid visualization in applications like QGIS or Panoply.

In [ ]:
import pathlib as pl

import flopy
import matplotlib.pyplot as plt
import xarray as xa
import xugrid as xu

# Load the base watershed model

We will use the base watershed model to demonstrate MODFLOW NetCDF output functionality.  

In [ ]:
name = "watershed"
sim = flopy.mf6.MFSimulation.load(
    sim_name=name, sim_ws=pl.Path("../../data/watershed/")
)

# Modify the model to save NetCDF output

We need to set the `start_date_time` variable in the TDIS file and define the `export_netcdf` variable in the groundwater flow model `name_file`. We will specify `ugrid` NetCDF output. We'll turn on `DIS EXPORT_ARRAY_NETCDF` so we can visualize that data as well. 

In [ ]:
sim.tdis.start_date_time = "1980-01-01"

In [ ]:
gwf = sim.get_model()
gwf.export_netcdf = "ugrid"
gwf.dis.export_array_netcdf = True

# Define `WKT` string and create `NCF` package

The `NCF` package is currently a subpackage of the model discretization (`DIS` or `DISV`) package.

In [ ]:
wkt = """PROJCRS["NAD83 / UTM ZONE 18N",BASEGEOGCRS["NAD83",DATUM["NORTH AMERICAN DATUM 1983",ELLIPSOID["GRS 1980",6378137,298.257222101,LENGTHUNIT["METRE",1]]],PRIMEM["GREENWICH",0,ANGLEUNIT["DEGREE",0.0174532925199433]],ID["EPSG",4269]],CONVERSION["UTM ZONE 18N",METHOD["TRANSVERSE MERCATOR",ID["EPSG",9807]],PARAMETER["LATITUDE OF NATURAL ORIGIN",0,ANGLEUNIT["DEGREE",0.0174532925199433],ID["EPSG",8801]],PARAMETER["LONGITUDE OF NATURAL ORIGIN",-75,ANGLEUNIT["DEGREE",0.0174532925199433],ID["EPSG",8802]],PARAMETER["SCALE FACTOR AT NATURAL ORIGIN",0.9996,SCALEUNIT["UNITY",1],ID["EPSG",8805]],PARAMETER["FALSE EASTING",500000,LENGTHUNIT["METRE",1],ID["EPSG",8806]],PARAMETER["FALSE NORTHING",0,LENGTHUNIT["METRE",1],ID["EPSG",8807]]],CS[CARTESIAN,2],AXIS["(E)",EAST,ORDER[1],LENGTHUNIT["METRE",1]],AXIS["(N)",NORTH,ORDER[2],LENGTHUNIT["METRE",1]],USAGE[SCOPE["ENGINEERING SURVEY, TOPOGRAPHIC MAPPING."],AREA["NORTH AMERICA - BETWEEN 78°W AND 72°W - ONSHORE AND OFFSHORE. CANADA - NUNAVUT; ONTARIO; QUEBEC. UNITED STATES (USA) - CONNECTICUT; DELAWARE; MARYLAND; MASSACHUSETTS; NEW HAMPSHIRE; NEW JERSEY; NEW YORK; NORTH CAROLINA; PENNSYLVANIA; VIRGINIA; VERMONT."],BBOX[28.28,-78,84,-72]],ID["EPSG",26918]]"""

ncf = flopy.mf6.ModflowUtlncf(
    gwf.dis,
    ogc_wkt=wkt,
    filename="model.dis.ncf",
)

# Change the simulation workspace, write the model files, and run the model

In [ ]:
ws = pl.Path("temp/step2")
sim.set_sim_path(ws)

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()